In [1]:
import sys
sys.path.append('../')

import pyterrier as pt
import pandas as pd
import os
from src.create_index import load_data, load_folds

In [2]:
sub_collections = [
    "t1",
    "t2",
    "t3",
    "t4",
    "t5",
]

In [3]:
def load_runs_for_subcollection(runs_path, sub_collection, fold_no):
    run_files = os.listdir(runs_path)
    runs = []
    run_names = []
    for run_file in run_files:
        if "_"+sub_collection in run_file and "F"+fold_no in run_file:
            run_name = run_file
            if "extended" in run_name or "long" in run_name:
                continue
        
            run_names.append(run_name)    
            run = pt.io.read_results(os.path.join(runs_path, run_name))
            runs.append(run)
    
    return runs, run_names

In [4]:
def filter_known_topics(df, sub_collection):
    sub_collections = ["t0", "t1", "t2", "t3", "t4", "t5"]
    query_map = pd.read_csv("../data/query_id_map.csv.gz", compression="gzip")
    query_map = query_map.dropna(subset=[sub_collection]).set_index(sub_collection)
    history = sub_collections[: sub_collections.index(sub_collection)]
    qids = query_map[history].dropna(how="all")
    qids = set(qids.index.tolist())
    return df[df["qid"].isin(qids)]

In [5]:
def make_table(results, folds=False):
    table = results.copy()
    
    
    # Rename runs
    table['name'] = table['name'].str.split('_').str[0]
    table = table.replace({"BM25": "BM25", "BM25+RM3": "+RM3", "BM25+Bo1": "+Bo1", "BM25+RF":"+RF", "BM25+qrel": "+qrel"})

    table = table.rename(columns={"name": "Run", "sub_collection":"t", "ndcg":"nDCG", "ndcg_cut.10":"nDCG@10", "P.10": "P@10", "recip_rank": "MRR"})
    table = table.replace({"t1": "$t_1$", "t2": "$t_2$", "t3": "$t_3$", "t4": "$t_4$", "t5": "$t_5$"})
    
    custom_order = ['BM25', '+RM3', '+Bo1', '+RF', '+qrel']
    table['Run'] = pd.Categorical(table['Run'], categories=custom_order, ordered=True)
    
    # Average over folds    
    if folds:
        table = table.drop("fold_no", axis=1).groupby(["t", "Run"]).mean()
    else:
        table = table.groupby(["t", "Run"]).mean()
           
    return table

# Natural Evolving

In [6]:
results = pd.DataFrame()

for sub_collection in sub_collections:
    topics, qrels = load_data(sub_collection)
    runs, run_names = load_runs_for_subcollection("../data/results_full", sub_collection, "0")
    topics = filter_known_topics(topics, sub_collection)

    res = pt.Experiment(
        runs,
        topics,
        qrels,
        names=run_names,
        eval_metrics=["ndcg", "ndcg_cut.10", "bpref", "P.10", "recip_rank"],
        verbose=True
        )

    res["sub_collection"] = sub_collection
    results = pd.concat([results, res])

Java started (triggered by _read_topics_trec) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
pt.Experiment: 100%|██████████| 5/5 [00:13<00:00,  2.75s/system]


In [7]:
table = make_table(results)
table

/tmp/ipykernel_232875/1755490063.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  table = table.groupby(["t", "Run"]).mean()


nDCG   nDCG@10     bpref      P@10       MRR
t     Run                                                    
$t_1$ BM25   0.290020  0.167360  0.322343  0.097083  0.267963
      +RM3   0.289160  0.160526  0.331976  0.097917  0.248290
      +Bo1   0.298634  0.172203  0.338289  0.100833  0.266395
      +RF    0.459869  0.344109  0.511598  0.149583  0.577352
      +qrel  0.418836  0.328013  0.388725  0.152917  0.498002
$t_2$ BM25   0.323675  0.198455  0.344076  0.127331  0.288200
      +RM3   0.323631  0.193317  0.342006  0.127010  0.282360
      +Bo1   0.331870  0.199886  0.348590  0.131190  0.287696
      +RF    0.500509  0.376979  0.506636  0.194212  0.616894
      +qrel  0.445316  0.348833  0.409334  0.190193  0.535177
$t_3$ BM25   0.334860  0.163117  0.406156  0.145226  0.308580
      +RM3   0.334269  0.151833  0.424149  0.138191  0.274880
      +Bo1   0.336104  0.161235  0.420819  0.147236  0.285041
      +RF    0.448392  0.266246  0.531198  0.213568  0.563914
      +qrel  0.393518  0.237447  0.440970  0.192965  0.456508
$t_4$ BM25   0.298771  0.171775  0.386605  0.145455  0.326048
      +RM3   0.300090  0.171134  0.406958  0.151196  0.305384
      +Bo1   0.307143  0.176746  0.404708  0.152632  0.326422
      +RF    0.397164  0.272669  0.471980  0.203828  0.592867
      +qrel  0.413769  0.321369  0.427354  0.229665  0.607544
$t_5$ BM25   0.214618  0.132661  0.290880  0.109076  0.300414
      +RM3   0.219670  0.133551  0.299115  0.111021  0.293259
      +Bo1   0.217928  0.132375  0.297316  0.112318  0.285249
      +RF    0.282513  0.201998  0.344549  0.154619  0.480281
      +qrel  0.319701  0.272991  0.324258  0.201135  0.607321

In [8]:
table.reset_index().to_latex("../paper/table-results.tex", index=False, float_format="%.3f")

# Cross Validation

In [13]:
results = pd.DataFrame()

folds = load_folds()

for sub_collection in sub_collections:    
    topics, qrels = load_data(sub_collection)
    topics = filter_known_topics(topics, sub_collection)
        
    for fold_no in range(0, len(folds[sub_collection])):
        train_docids = folds[sub_collection][str(fold_no)]["train"]
        test_docids = folds[sub_collection][str(fold_no)]["test"]
                    
        runs, run_names = load_runs_for_subcollection("../data/results", sub_collection, str(fold_no))
    
        qrels_fold = qrels.copy()
        qrels_fold = qrels_fold[~qrels_fold["docno"].isin(train_docids)]
        
        res = pt.Experiment(
        runs,
        topics,
        qrels,
        names=run_names,
        eval_metrics=["ndcg", "ndcg_cut.10", "bpref", "P.10", "recip_rank"],
        verbose=True
        )
        
        res["sub_collection"] = sub_collection
        res["fold_no"] = fold_no
        results = pd.concat([results, res])

pt.Experiment: 100%|██████████| 5/5 [00:13<00:00,  2.68s/system]


In [14]:
table = make_table(results, folds=True)
table 

/tmp/ipykernel_232875/1755490063.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  table = table.drop("fold_no", axis=1).groupby(["t", "Run"]).mean()


nDCG   nDCG@10     bpref      P@10       MRR
t     Run                                                    
$t_1$ BM25   0.210387  0.124980  0.228038  0.069028  0.217848
      +RM3   0.209302  0.120403  0.233044  0.070139  0.206548
      +Bo1   0.217031  0.130342  0.238136  0.072639  0.219205
      +RF    0.328842  0.251247  0.346069  0.114306  0.516901
      +qrel  0.365628  0.311513  0.314509  0.130972  0.598246
$t_2$ BM25   0.257668  0.160785  0.268939  0.101072  0.253991
      +RM3   0.255681  0.156390  0.266143  0.100857  0.245758
      +Bo1   0.261900  0.159913  0.271265  0.102465  0.252486
      +RF    0.358357  0.262092  0.351168  0.140514  0.511538
      +qrel  0.381505  0.308861  0.342533  0.157878  0.557248
$t_3$ BM25   0.261485  0.133750  0.315836  0.116918  0.275848
      +RM3   0.259608  0.123261  0.328658  0.110553  0.244834
      +Bo1   0.260430  0.129971  0.325410  0.118258  0.250218
      +RF    0.345672  0.221157  0.392566  0.169514  0.542792
      +qrel  0.334116  0.222856  0.360905  0.163652  0.507887
$t_4$ BM25   0.234039  0.138516  0.306403  0.115949  0.286596
      +RM3   0.235974  0.140631  0.320773  0.122807  0.272655
      +Bo1   0.241687  0.145484  0.320737  0.123445  0.292993
      +RF    0.312474  0.229445  0.359845  0.162998  0.542463
      +qrel  0.342704  0.276755  0.354069  0.189314  0.605949
$t_5$ BM25   0.184734  0.116216  0.250507  0.093679  0.276000
      +RM3   0.189237  0.117816  0.257660  0.096110  0.271239
      +Bo1   0.186812  0.116100  0.256194  0.097083  0.260528
      +RF    0.225393  0.161977  0.276389  0.118801  0.419137
      +qrel  0.239583  0.187248  0.268555  0.134522  0.471664

In [15]:
# table.to_latex("../paper/table-results-fold.tex", float_format="%.3f")
table.reset_index().to_latex("../paper/table-results-fold.tex", index=False, float_format="%.3f")